<h1><a href="https://arxiv.org/abs/1802.08678">
Verifying Controllers Against Adversarial Examples with Bayesian Optimization</a></h1>
Shromona Ghosh et al.

<h2>Summary</h2>

* Use boolean combinations of smooth functions on the trajectories as safety specification

* Use Baysesian Optimization to actively test the controller to find adversarial counterexamples

<h2>Motivation</h2>

* Conventional techniques in designing robust controllers rely on simple linear models of the underlying system.
    * Either overly conservative
    
    * Or inaccurate due to overapproximation, e.g. cannot capture nonlinear effects
    
* Reinforcement learning can generate high fidelity controllers
    * No formal guarantees for safety

* Formal safety certificates by using formal methods
    * Curse of dimensionality
    * Only use simple system dynamics
    
* Falsification tests the system in various environments seeking for adversarial examples
    * Perturbations must be meaningful for dynamic systems
* Test black-box systems by using smart search techinques
    * Sequential search algorithms based on heuristics, e.g. CMA-ES, Simulated Annearling, does not utilize information of previous simulations


<h2>Problem Formulation</h2>

* A closed-loop system uses a model parameterizing environment uncertainty with $w\in\mathbb{W}$

* If the system remain safe under all uncertain scenarios, then it satisfiies the safety specfication $\forall w\in\mathbb{W}, \varphi(w)>0$

* Test wether there is an adversarial example $w\in\mathbb{W}$ s.t. $\varphi(w)<0$ and minimize the test cost, e.g. number of simulations
$$argmin_{w\in\mathbb{W}} \varphi(w)$$

* Key problem is that functional dependence $\varphi(w)$ and $w$ is unknown.

<h2>Background</h2>

* Safety Specification
$$\varphi:= \mu|\neg\mu|\varphi_1\vee\varphi_2|\varphi_1\wedge\varphi_2$$
where $\mu:\Xi\rightarrow\mathbb{R}$ returns the 'robustness' of a trajectory $\xi\in\Xi$

$$\mu(\xi):=\mu(\xi), (\varphi_1\vee\varphi_2)(\xi):=min(\varphi_1, \varphi_2)),\\
  \neg\mu(\xi):=-\mu(\xi), (\varphi_1\wedge\varphi_2):=max(\varphi_1, \varphi_2)),$$
  
* Gaussian Process
    * $\hat{\\\mu}(w)$ is the observed target function $\mu(w)$ with Gaussian noise $\hat{\\\mu}(w)=\mu(w)+\omega$ where $\omega\sim\mathcal{N}(0,\sigma^2)$.
    * Given $n$ observation $y_n=(\hat{\\\mu}(w_1), \hat{\\\mu}(w_2),\ldots, \hat{\\\mu}(w_n))$ and their corresponding input $W_n=\{w_1, w_2,\ldots, w_n\}$, the posterior over function $\mu(w)$ has expectation $m_n(w)$, covariance $k_n(w,w) and variance $\sigma_n(w)$ where

\begin{eqnarray}
m_n(w)&=&k_n(w)(K_n+I_n\sigma^2)^{-1}y_n\\
k_n(w,w')&=&k(w,w')-k_n(w)(K_n+I_n\sigma^2)^{-1}k^T_n(w')\\
\sigma^2_n(w)&=&k_n(w,w')\\
\\
k_n(w)&=&[k(w,w_1),\ldots, k(w,w_n)]\\
[K_n](i,j)&=&k(w_i,w_j)
\end{eqnarray}


* Bayesian Optimization
    * The optimization function based on GP is $w_n=argmin_{w\in\mathbb{W}} m_{n-1}(w)-\beta^{1/2}_n\sigma_{n-1}(w)$


<h2>Approach</h2>


* `Parse Tree` $\mathcal{T}$: given a specification formula $\varphi$, the corresponding parse tree $\mathcal{T}$ has leaf nodes that corresponding to function predicates(atomics), while other nodes are max(disjunctions) and min(conjunctions) (and negation????). Use the quantified specification to represent the tree

\begin{eqnarray}
\varphi(w)&=&(\mu_1\wedge\mu_2)\vee\ldots\\
&\Rightarrow& max(min\ldots)\\
\mathcal{T}(\mu_1(w),\ldots, \mu_q(w))&=&\varphi(w) 
\end{eqnarray}

* The distribution of the tree is bounded with high probability by the lower-confidence interval of one of the predicates. Consider the **lower bounds** within the confidence intervals of each $\mu_i$ in the tree 
$$l_1=m^i_{n-1}(w)-\beta_n^{1/2}\sigma^i_{n-1}(w)$$

* Then a heuristic solution to BO optimization is to find a $w$ that maximally approaching the lower bounds of the confidence intervals of all the predicates. 
$$w=argmin_{w\in\mathbb{W}} \varphi(l_1(w),l_2(w),\ldots,l_q(w))$$
After $w$ is solved, add $w$ back to $W$ and start next propagation


